In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [17]:
# Common imports
import fitsio
import numpy as np
import os
import pandas as pd
import sys
import glob

In [3]:
# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.rc('font', size=18)

In [4]:
# Tensorflow imports
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [5]:
# QuasarNET imports
from quasarnet.models import QuasarNET, custom_loss
from quasarnet.io import read_truth, read_data, objective
from quasarnet.utils import process_preds, absorber_IGM

In [6]:
from astropy.io import fits
from astropy.table import Table, join, vstack, hstack

In [7]:
# To make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.compat.v1.reset_default_graph
    tf.random.set_seed(seed)
    np.random.seed(seed)

In [8]:
def plot_spectrum(ival,X,c_line,z_line,zbest,Y,z,c_th=0.5,ndetect=1):
    llmin = np.log10(3600)
    llmax = np.log10(10000)
    dll = 1e-3

    nbins = int((llmax-llmin)/dll)
    wave = 10**(llmin + np.arange(nbins)*dll)
    fig, ax = plt.subplots(1,1,figsize=(10,6))
    ax.plot(wave, X[ival,:])
    
    isqso_truth = (Y[ival,:].argmax()==2) | (Y[ival,:].argmax()==3)
    isqso_qn = (c_line[:,ival].sum()>c_th)>=ndetect
    
    title = r'Is QSO? VI: {}, QN: {}'.format(isqso_truth,isqso_qn)
    title += '\n'
    title += r'$z_{{VI}}$={:1.3f}, $z_{{QN}}$='.format(z[ival])
    if isqso_qn:
        title += r'{:1.3f}'.format(zbest[ival])
    else:
        title += 'N/A'
    
    ax.set_title(title)
    m = X[ival,:].min()
    M = X[ival,:].max()
    ax.grid()
    ax.set_ylim(m-2,M+2)
    for il,l in enumerate(lines):
        lam = absorber_IGM[l]*(1+z_line[il,ival])
        w = abs(wave-lam)<100
        if w.sum()>0:
            m = X[ival,w].min()-1
            M = X[ival,w].max()+1
            ax.plot([lam,lam], [m,M],'r--', alpha=0.1+0.9*c_line[il,ival])
            ax.text(lam,M+0.5,'c$_{{{}}}={}$'.format(l,round(c_line[il,ival],3)),
                     horizontalalignment='center',alpha=0.1+0.9*c_line[il,ival])
    ax.set_xlabel(r'$\lambda_\mathrm{obs}~[\AA]$')
    ax.set_ylabel(r'renormalised flux')
    plt.show()
    
    return

# Get blanc deep tile data, and run QuasarNET:

### parse_data_desisim --spectra /global/cfs/cdirs/desi/spectro/redux/blanc/tiles/80609/deep/coadd*.fits --out blanc-80606-deep.fits.gz --mode DESI_COADD --desi-period sv
### qn_export --model /global/cfs/projectdirs/desi/users/jfarr/QuasarNET_paper/qn_models/main_setup/coadd/prop_0.1/model_indtrain_0_0/qn_train_coadd_indtrain_0_0.h5 --data blanc-80609-deep.fits.gz --mode DESI_COADD --out-suffix output-806023-deep --data-training /global/cfs/projectdirs/desi/users/jfarr/QuasarNET_paper/data/coadd/training_datasets/prop_0.1/data_dr12_coadd_train_indtrain_0_0.fits 

# Get cascades data, and run QuasarNET:

### parse_data_desisim --spectra /global/cfs/cdirs/desi/spectro/redux/cascades/tiles/80609/deep/coadd*.fits --out cascades-80609-deep.fits.gz --mode DESI_COADD --desi-period sv
### qn_export --model /global/cfs/projectdirs/desi/users/jfarr/QuasarNET_paper/qn_models/main_setup/coadd/prop_0.1/model_indtrain_0_0/qn_train_coadd_indtrain_0_0.h5 --data cascades-80609-deep.fits.gz --mode DESI_COADD --out-suffix output-80609-cascades --data-training/global/cfs/projectdirs/desi/users/jfarr/QuasarNET_paper/data/coadd/training_datasets/prop_0.1/data_dr12_coadd_train_indtrain_0_0.fits 


# Loading results

In [9]:
def load_qn_ting_tile(tile,release):
    """Return QuasarNET results for a particular tile"""
    basedir='/global/homes/t/tanting/Machine_Learning_quasar/QuasarNET/test_qn_blanc_ting/'
    if release == 'blanc':
        fname=basedir+'qnAll-output-{}-deep.fits'.format(tile,release)
        fname_spectra=basedir+'{}-{}-deep.fits.gz'.format(release,tile)
    elif release == 'cascades':
        fname=basedir+'qnAll-output-{}-{}.fits'.format(tile,release)
        fname_spectra=basedir+'{}-{}-deep.fits.gz'.format(release,tile)
    hdul = fits.open(fname)
    data = Table(hdul[1].data)
    # remove columns that are not needed
    data.remove_columns(['IN_TRAIN','FIBER','BRICKNAME','BRICK_OBJID','Z_LINES_BAL','C_LINES_BAL'])
    data.rename_column('ZBEST', 'QN_Z')
    data.rename_column('IS_QSO', 'QN_IS_QSO')   
    # add spectra
    #fname=basedir+'blanc-{}.fits.gz'.format(tile)
    
    hdul_spectra = fits.open(fname_spectra)
    target_spectra = Table(hdul_spectra[1].data)
    target_spectra.remove_columns(['SPID0','SPID1','SPID2'])
    spectra = hdul_spectra[0].data[:,:443]
    spectra[:,434:443]=0
    target_spectra.add_column(spectra, name='spectra')
    data = join(data,target_spectra,keys='TARGETID')
    return data


In [10]:
qn_80605_blanc=load_qn_ting_tile('80605','blanc')
qn_80607_blanc=load_qn_ting_tile('80607','blanc')
qn_80609_blanc=load_qn_ting_tile('80609','blanc')

In [11]:
qn_80605_cascades=load_qn_ting_tile('80605','cascades')
qn_80607_cascades=load_qn_ting_tile('80607','cascades')
qn_80609_cascades=load_qn_ting_tile('80609','cascades')

In [12]:
def load_wvi_tile(fname='80605_QSOzinfo_wVI.fits'):
    """Return VI truth table for a particular tile"""
    basedir='/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/blanc/v2/QSO/'
    data=Table.read(basedir+fname)
    # remove columns that are not needed
    data=data['TARGETID','Z','best_z','best_quality','SPECTYPE','best_spectype']
    data.rename_column('Z', 'RR_Z')
    data.rename_column('SPECTYPE', 'RR_CLASS')
    data.rename_column('best_z', 'VI_Z')
    data.rename_column('best_spectype', 'VI_CLASS')
    data.rename_column('best_quality', 'VI_QUALITY')
    return data

In [13]:
def load_vi(fname,release,vi_type):
    """Return VI truth table for a particular tile"""
    if release == 'blanc':
        basedir='/global/cfs/cdirs/desi/sv/vi/TruthTables/Blanc/QSO/'
    elif release == 'cascades':
        basedir='/global/cscratch1/sd/ajross/SV1/redshift_comps/cascades/testcas/QSO/'
    if vi_type == 'vi':
        data=Table.read(basedir+fname)
        # remove columns that are not needed
        data.remove_columns(['all_VI_issues','all_VI_comments','merger_comment','N_VI'])
        data.rename_column('Redrock_z', 'RR_Z')
        data.rename_column('Redrock_spectype', 'RR_CLASS')
        data.rename_column('best_z', 'VI_Z')
        data.rename_column('best_spectype', 'VI_CLASS')
        data.rename_column('best_quality', 'VI_QUALITY')
    elif vi_type == 'wvi':
        basedir='/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/blanc/v2/QSO/'
        data=Table.read(basedir+fname)
        # remove columns that are not needed
        data=data['TARGETID','Z','best_z','best_quality','SPECTYPE','best_spectype']
        data.rename_column('Z', 'RR_Z')
        data.rename_column('SPECTYPE', 'RR_CLASS')
        data.rename_column('best_z', 'VI_Z')
        data.rename_column('best_spectype', 'VI_CLASS')
        data.rename_column('best_quality', 'VI_QUALITY')
    return data

In [14]:
vi_80605_blanc=load_vi('desi-vi_QSO_tile80605_nightdeep_merged_all_210223.csv','blanc','vi')
vi_80607_blanc=load_vi('desi-vi_QSO_tile80607_nightdeep_merged_all_210214.csv','blanc','vi')
vi_80609_blanc=load_vi('desi-vi_QSO_tile80609_nightdeep_merged_all_210210.csv','blanc','vi')

In [15]:
vi_80605_cascades=load_vi('80605_QSOzinfo_wVI.fits','cascades','wvi')
vi_80607_cascades=load_vi('80607_QSOzinfo_wVI.fits','cascades','wvi')
vi_80609_cascades=load_vi('80609_QSOzinfo_wVI.fits','cascades','wvi')

# Get SQ results

In [35]:
def merge_data(files,columns=None):
    merged_data=[]
    for file in files:
        merged_data.append(Table(fitsio.read(file,columns=columns)))
    return vstack(merged_data)

In [70]:
def load_SQ(fname,release,release_type,tile):
    sqfiles_path=fname+'/'+release
    squeze_model = 'boss'
    sqfiles=sorted(glob.glob(os.path.join(sqfiles_path,release_type,tile,f'{squeze_model}*.fits')))
    sq_table=merge_data(sqfiles,columns=['TARGETID','PROB','Z_TRY','CLASS_PREDICTED'])
    sq_table.rename_column('Z_TRY','SQ_Z')
    sq_table.rename_column('CLASS_PREDICTED','SQ_CLASS')
    return sq_table
    

In [72]:
sq_80605_blanc = load_SQ('/global/cfs/cdirs/desi/users/hiramk/desi/squeze_tests','blanc','tiles','80605')
sq_80607_blanc = load_SQ('/global/cfs/cdirs/desi/users/hiramk/desi/squeze_tests','blanc','tiles','80607')
sq_80609_blanc = load_SQ('/global/cfs/cdirs/desi/users/hiramk/desi/squeze_tests','blanc','tiles','80609')
sq_80605_cascades = load_SQ('/global/cfs/cdirs/desi/users/hiramk/desi/squeze_tests','cascades','deep','80605')
sq_80607_cascades = load_SQ('/global/cfs/cdirs/desi/users/hiramk/desi/squeze_tests','cascades','deep','80607')
sq_80609_cascades = load_SQ('/global/cfs/cdirs/desi/users/hiramk/desi/squeze_tests','cascades','deep','80609')

# Merge VI tables with QN results

In [73]:
vi_qn_sq_80605_blanc=join(join(vi_80605_blanc,qn_80605_blanc,keys='TARGETID'),sq_80605_blanc,keys='TARGETID')
vi_qn_sq_80607_blanc=join(join(vi_80607_blanc,qn_80607_blanc,keys='TARGETID'),sq_80607_blanc,keys='TARGETID')
vi_qn_sq_80609_blanc=join(join(vi_80609_blanc,qn_80609_blanc,keys='TARGETID'),sq_80609_blanc,keys='TARGETID')
# combine tiles
vi_qn_sq_all_blanc=vstack([vi_qn_sq_80605_blanc,vstack([vi_qn_sq_80607_blanc,vi_qn_sq_80609_blanc])])

In [74]:
vi_qn_sq_80605_cascades=join(join(vi_80605_blanc,qn_80605_cascades,keys='TARGETID'),sq_80605_cascades,keys='TARGETID')
vi_qn_sq_80607_cascades=join(join(vi_80607_blanc,qn_80607_cascades,keys='TARGETID'),sq_80607_cascades,keys='TARGETID')
vi_qn_sq_80609_cascades=join(join(vi_80609_blanc,qn_80609_cascades,keys='TARGETID'),sq_80609_cascades,keys='TARGETID')
# combine tiles
vi_qn_sq_all_cascades=vstack([vi_qn_sq_80605_cascades,vstack([vi_qn_sq_80607_cascades,vi_qn_sq_80609_cascades])])

In [91]:
def print_summary(data,min_qual=2.5,min_z=0.0,min_prob=0.0):
    n_tot=len(data)
    # only objects with good VI quality
    w_qual=data['VI_QUALITY']>=min_qual
    n_qual=len(data[w_qual])
    print('{:.3f} = ({} / {}) fraction of objects with VI quality >= {}'.format(n_qual/n_tot,n_qual,n_tot,min_qual))
    # objects identified as QSO by VI / Redrock / QuasarNET
    w_VI=(data['VI_CLASS']=='QSO') & (data['VI_Z'] > min_z)
    w_RR=data['RR_CLASS']=='QSO'
    #w_SQ=((data['SQ_CLASS']==3)|(data['SQ_CLASS']==30)|(data['SQ_CLASS']==35)|(data['SQ_CLASS']==305))&(data['PROB']>min_prob)
    w_SQ=((data['SQ_CLASS']==3)|(data['SQ_CLASS']==30))&(data['PROB']>min_prob)
    w_QN=data['QN_IS_QSO']==True
    # matches for good spectra
    n_VI=len(data[w_qual & w_VI])
    print('{:.3f} = ({} / {}) fraction of good VI spectra are quasars'.format(n_VI/n_qual,n_VI,n_qual))    
    n_RR=len(data[w_qual & w_VI & w_RR])
    print('{:.3f} = ({} / {}) fraction of good VI quasars identified by Redrock'.format(n_RR/n_VI,n_RR,n_VI))    
    n_QN=len(data[w_qual & w_VI & w_QN])
    print('{:.3f} = ({} / {}) fraction of good VI quasars identified by QuasarNET'.format(n_QN/n_VI,n_QN,n_VI))
    n_SQ=len(data[w_qual & w_VI & w_SQ])
    print('{:.3f} = ({} / {}) fraction of good VI quasars identified by Squeeze'.format(n_SQ/n_VI,n_SQ,n_VI))
    n_intersection=len(data[w_qual & w_VI & w_RR & w_QN & w_SQ])
    print('{:.3f} = ({} / {}) fraction of good VI quasars identified by both Redrock, Squeeze and QuasarNET'.format(n_intersection/n_VI,n_intersection,n_VI))
    n_union=len(data[(w_qual & w_VI & w_RR)|(w_qual & w_VI & w_QN)|(w_qual & w_VI & w_SQ)])
    print('{:.3f} = ({} / {}) fraction of good VI quasars identified by either Redrock, Squeeze or QuasarNET'.format(n_union/n_VI,n_union,n_VI))
    
    w_high_z = (data['VI_Z'] > min_z)
    n_RR_total = len(data[w_qual & w_RR & w_high_z])
    n_RR_false=len(data[w_qual & ~w_VI & w_RR & w_high_z])
    print('{:.3f} = ({} / {}) fraction of VI targets false identified by Redrock as quasars'.format(n_RR_false/n_RR_total,n_RR_false,n_RR_total)) 
    
    n_QN_total = len(data[w_qual & w_QN & w_high_z])
    n_QN_false=len(data[w_qual & ~w_VI & w_QN & w_high_z])
    print('{:.3f} = ({} / {}) fraction of VI targets false identified by QuasarNET as quasars'.format(n_QN_false/n_QN_total,n_QN_false,n_QN_total)) 
    
    #n_SQ_total = len(data[w_qual & w_SQ & w_high_z])
    #n_SQ_false=len(data[w_qual & ~w_VI & w_SQ & w_high_z])
    #print('{:.3f} = ({} / {}) fraction of VI targets false identified by Squeeze as quasars'.format(n_SQ_false/n_SQ_total,n_SQ_false,n_SQ_total)) 

In [92]:

#print('tile 80605')
#print_summary(data=vi_qn_80605,min_qual=2.5)
#print('tile 80607')
#print_summary(data=vi_qn_80607,min_qual=2.5)
#print('tile 80609')
#print_summary(data=vi_qn_80609,min_qual=2.5)
print('blanc all tiles')
print_summary(data=vi_qn_sq_all_blanc,min_qual=2.5)
print('cascades all tiles')
print_summary(data=vi_qn_sq_all_cascades,min_qual=2.5)

blanc all tiles
0.873 = (2962 / 3392) fraction of objects with VI quality >= 2.5
0.421 = (1248 / 2962) fraction of good VI spectra are quasars
0.854 = (1066 / 1248) fraction of good VI quasars identified by Redrock
0.948 = (1183 / 1248) fraction of good VI quasars identified by QuasarNET
0.898 = (1121 / 1248) fraction of good VI quasars identified by Squeeze
0.800 = (999 / 1248) fraction of good VI quasars identified by both Redrock, Squeeze and QuasarNET
0.974 = (1216 / 1248) fraction of good VI quasars identified by either Redrock, Squeeze or QuasarNET
0.003 = (3 / 1069) fraction of VI targets false identified by Redrock as quasars
0.038 = (47 / 1230) fraction of VI targets false identified by QuasarNET as quasars
cascades all tiles
0.900 = (62355 / 69259) fraction of objects with VI quality >= 2.5
0.507 = (31635 / 62355) fraction of good VI spectra are quasars
0.866 = (27403 / 31635) fraction of good VI quasars identified by Redrock
0.972 = (30749 / 31635) fraction of good VI quasar

# Results for quasars at z>2.1

In [81]:
#print('tile 80605')
#print_summary(data=vi_qn_80605,min_qual=2.5,min_z=2.1)
#print('tile 80607')
#print_summary(data=vi_qn_80607,min_qual=2.5,min_z=2.1)
#print('tile 80609')
#print_summary(data=vi_qn_80609,min_qual=2.5,min_z=2.1)
print('blanc all tiles')
print_summary(data=vi_qn_sq_all_blanc,min_qual=2.5,min_z=2.1)
print('cascades all tiles')
print_summary(data=vi_qn_sq_all_cascades,min_qual=2.5,min_z=2.1)

blanc all tiles
0.873 = (2962 / 3392) fraction of objects with VI quality >= 2.5
0.123 = (364 / 2962) fraction of good VI spectra are quasars
0.915 = (333 / 364) fraction of good VI quasars identified by Redrock
0.981 = (357 / 364) fraction of good VI quasars identified by QuasarNET
0.992 = (361 / 364) fraction of good VI quasars identified by Squeeze
0.898 = (327 / 364) fraction of good VI quasars identified by both Redrock, Squeeze and QuasarNET
1.000 = (364 / 364) fraction of good VI quasars identified by either Redrock, Squeeze or QuasarNET
0.003 = (1 / 334) fraction of VI targets false identified by Redrock as quasars
0.011 = (4 / 361) fraction of VI targets false identified by QuasarNET as quasars
cascades all tiles
0.900 = (62355 / 69259) fraction of objects with VI quality >= 2.5
0.167 = (10420 / 62355) fraction of good VI spectra are quasars
0.915 = (9530 / 10420) fraction of good VI quasars identified by Redrock
0.990 = (10315 / 10420) fraction of good VI quasars identified b

# Contaminants in reobservations
## Here only consider about classification, not about z precision

In [88]:
def reobs_info(data,min_qual=2.5,min_prob=0):
    n_tot=len(data)
    # only objects with good VI quality
    w_qual=data['VI_QUALITY']>=min_qual
    n_qual=len(data[w_qual])
    print('{:.3f} = ({} / {}) objects with VI quality >= {}'.format(n_qual/n_tot,n_qual,n_tot,min_qual))
    # true high-z quasars found during VI
    w_VI=(data['VI_CLASS']=='QSO') & (data['VI_Z'] >= 2.1)
    n_VI=len(data[w_VI & w_qual])
    print('{:.3f} = ({} / {}) VIed QSOs with z >= 2.1'.format(n_VI/n_qual,n_VI,n_qual))
    
    # objects selected by Redrock for reobservation 
    w_RR=(data['RR_CLASS']=='QSO') & (data['RR_Z'] >= 2.1)
    n_RR=len(data[w_RR & w_qual])
    print('{} objects selected by RR for re-observation'.format(n_RR))
    n_RR_good=len(data[w_RR & w_qual & w_VI])
    print('{:.3f} = ({} / {}) completeness in RR selection for re-observation'.format(n_RR_good/n_VI,n_RR_good,n_VI))
    print('{:.3f} = ({} / {}) purity in RR selection for re-observation'.format(n_RR_good/n_RR,n_RR_good,n_RR))
    
    # objects selected by QuasarNET for reobservation 
    w_QN=(data['QN_IS_QSO']==True) & (data['QN_Z'] >= 2.1)
    n_QN=len(data[w_QN & w_qual])
    print('{} objects selected by QN for re-observation'.format(n_QN))
    n_QN_good=len(data[w_QN & w_qual & w_VI])
    print('{:.3f} = ({} / {}) completeness in QN selection for re-observation'.format(n_QN_good/n_VI,n_QN_good,n_VI))
    print('{:.3f} = ({} / {}) purity in QN selection for re-observation'.format(n_QN_good/n_QN,n_QN_good,n_QN))
    
    # objects selected by Squeeze for reobservation 
    #w_SQ=((data['SQ_CLASS']==3)|(data['SQ_CLASS']==30)|(data['SQ_CLASS']==35)|(data['SQ_CLASS']==305))&(data['PROB']>min_prob) & (data['SQ_Z'] >= 2.1)
    w_SQ=((data['SQ_CLASS']==3)|(data['SQ_CLASS']==30))&(data['PROB']>min_prob) & (data['SQ_Z'] >= 2.1)
    n_SQ=len(data[w_SQ & w_qual])
    print('{} objects selected by Squeeze for re-observation'.format(n_SQ))
    n_SQ_good=len(data[w_SQ & w_qual & w_VI])
    print('{:.3f} = ({} / {}) completeness in SQ selection for re-observation'.format(n_SQ_good/n_VI,n_SQ_good,n_VI))
    print('{:.3f} = ({} / {}) purity in SQ selection for re-observation'.format(n_SQ_good/n_SQ,n_SQ_good,n_SQ))
    
    w_QN_RR_SQ = w_RR|w_QN|w_SQ
    n_QN_RR_SQ = len(data[w_QN_RR_SQ & w_qual])
    n_QN_RR_SQ_good=len(data[w_QN_RR_SQ & w_qual & w_VI])
    print('{} objects selected by QN+RR for re-observation'.format(n_QN_RR_SQ))
    print('{:.3f} = ({} / {}) completeness in QN+RR+SQ selection for re-observation'.format(n_QN_RR_SQ_good/n_VI,n_QN_RR_SQ_good,n_VI))
    print('{:.3f} = ({} / {}) purity in QN+RR+SQ selection for re-observation'.format(n_QN_RR_SQ_good/n_QN,n_QN_RR_SQ_good,n_QN_RR_SQ))
    

In [89]:
#print('tile 80605')
#reobs_info(data=vi_qn_80605,min_qual=2.5)
#print('tile 80607')
#reobs_info(data=vi_qn_80607,min_qual=2.5)
#print('tile 80609')
#reobs_info(data=vi_qn_80609,min_qual=2.5)
print('all tiles blanc')
reobs_info(data=vi_qn_sq_all_blanc,min_qual=2.5)
print('all tiles cascades')
reobs_info(data=vi_qn_sq_all_cascades,min_qual=2.5)

all tiles blanc
0.873 = (2962 / 3392) objects with VI quality >= 2.5
0.123 = (364 / 2962) VIed QSOs with z >= 2.1
326 objects selected by RR for re-observation
0.885 = (322 / 364) completeness in RR selection for re-observation
0.988 = (322 / 326) purity in RR selection for re-observation
374 objects selected by QN for re-observation
0.973 = (354 / 364) completeness in QN selection for re-observation
0.947 = (354 / 374) purity in QN selection for re-observation
357 objects selected by Squeeze for re-observation
0.931 = (339 / 364) completeness in SQ selection for re-observation
0.950 = (339 / 357) purity in SQ selection for re-observation
401 objects selected by QN+RR for re-observation
0.995 = (362 / 364) completeness in QN+RR+SQ selection for re-observation
0.968 = (362 / 401) purity in QN+RR+SQ selection for re-observation
all tiles cascades
0.900 = (62355 / 69259) objects with VI quality >= 2.5
0.167 = (10420 / 62355) VIed QSOs with z >= 2.1
9377 objects selected by RR for re-obser

# Where is QuasarNET failing?

In [ ]:
def get_QN_misses(data,min_qual=2.5,min_z=2.1,error_z=0.05):
    """Return high-z quasars missed by QN"""
    # objects with good VI quality
    w_qual=data['VI_QUALITY']>=min_qual
    n_qual=len(data[w_qual])
    # high-z quasars found during VI
    w_VI_highz=(data['VI_CLASS']=='QSO') & (data['VI_Z'] >= min_z)
    n_VI_highz=len(data[w_VI_highz & w_qual])
    print('{} VIed QSOs with z >= 2.1'.format(n_VI_highz))
    # objects missed by QuasarNET (allow for some redshift error)
    #w_QN_nohighz=(data['QN_IS_QSO']==False) | (data['QN_Z'] <= min_z - error_z) | (data['QN_IS_QSO']==True and data['VI_CLASS']!='QSO') 
    w_QN_nohighz=(data['QN_IS_QSO']==False) | (min_z - error_z >= data['QN_Z'])
    QN_misses=data[w_qual & w_VI_highz & w_QN_nohighz]
    print('{} high-z quasars missed by QN'.format(len(QN_misses)))
    return QN_misses

In [ ]:
#print('tile = 80605')
#QN_misses_80605=get_QN_misses(data=vi_qn_80605,min_qual=2.5,min_z=2.1)
#print('tile = 80607')
#QN_misses_80607=get_QN_misses(data=vi_qn_80607,min_qual=2.5,min_z=2.1)
#print('tile = 80609')
#QN_misses_80609=get_QN_misses(data=vi_qn_80609,min_qual=2.5,min_z=2.1)
QN_misses=get_QN_misses(data=vi_qn_all,min_qual=2.5,min_z=2.1)

In [ ]:
QN_misses

In [ ]:
QN_misses[1]['C_LINES']

In [ ]:
QN_misses[1]['C_LINES']#.data

In [ ]:
QN_misses[1]['Z_LINES']#.data

In [ ]:
def get_QN_mistakes(data,min_qual=2.5,min_z=2.1):
    """Return high-z quasars from QN not confirmed by VI"""
    # objects with good VI quality
    w_qual=data['VI_QUALITY']>=min_qual
    n_qual=len(data[w_qual])
    # not high-z quasars found during VI
    w_VI_nohighz=(data['VI_CLASS']!='QSO') | (data['VI_Z'] <= min_z)
    # objects missed by QuasarNET (allow for some redshift error)
    w_QN_highz=(data['QN_IS_QSO']==True) & (data['QN_Z'] >= min_z)
    QN_mistakes=data[w_qual & w_VI_nohighz & w_QN_highz]
    print('{} non high-z quasars mistaken by QN'.format(len(QN_mistakes)))
    return QN_mistakes

In [ ]:
#print('tile = 80605')
#QN_mistakes_80605=get_QN_mistakes(data=vi_qn_80605,min_qual=2.5,min_z=2.1)
#print('tile = 80607')
#QN_mistakes_80607=get_QN_mistakes(data=vi_qn_80607,min_qual=2.5,min_z=2.1)
#print('tile = 80609')
#QN_mistakes_80609=get_QN_mistakes(data=vi_qn_80609,min_qual=2.5,min_z=2.1)
QN_mistakes=get_QN_mistakes(data=vi_qn_all,min_qual=2.5,min_z=2.1)

In [ ]:
QN_mistakes

In [ ]:
QN_mistakes[1]['C_LINES']#.data

In [ ]:
QN_mistakes[1]['Z_LINES']#.data

In [ ]:
def get_QN_false_z(data,min_qual=2.5,min_z=2.1,error_z=0.05):
    """Return high-z quasars classified by QN with wrong redshift"""
    # objects with good VI quality
    w_qual=data['VI_QUALITY']>=min_qual
    n_qual=len(data[w_qual])
    # high-z quasars found during VI
    w_VI_highz=(data['VI_CLASS']=='QSO') & (data['VI_Z'] >= min_z)
    n_VI_highz=len(data[w_VI_highz & w_qual])
    print('{} VIed QSOs with z >= 2.1'.format(n_VI_highz))
    # objects missed by QuasarNET (allow for some redshift error)
    w_QN_false_z=(data['QN_IS_QSO']==False) | (data['QN_Z'] >= data['VI_Z'] + error_z) | (data['VI_Z'] - error_z >= data['QN_Z'])
    QN_false_z=data[w_qual & w_VI_highz & w_QN_false_z]
    print('{} high-z quasars missed by QN'.format(len(QN_false_z)))
    return QN_false_z

In [ ]:
QN_false_z=get_QN_false_z(data=vi_qn_all,min_qual=2.5,min_z=2.1)

In [ ]:
QN_false_z

# Look at the spectra

In [ ]:
def plot_spectrum(tables):
    for table in tables:
        lines = ['LYA','CIV(1548)','CIII(1909)', 'MgII(2796)','Hbeta','Halpha']
        llmin = np.log10(3600)
        llmax = np.log10(10000)
        dll = 1e-3
        nbins = int((llmax-llmin)/dll)
        wave = 10**(llmin + np.arange(nbins)*dll)
        fig, ax = plt.subplots(1,1,figsize=(10,6))
        ax.plot(wave, table['spectra'])
        QN_CLASS = ''
        if table['QN_IS_QSO'] == np.int64(1):
            QN_CLASS = 'QSO'
        else:
            QN_CLASS = 'NO'
        title = r'TargetID:{}'.format(table['TARGETID'])
        title += '\n'
        title += r'VI:Z:{:1.3f},Class:{}'.format(table['VI_Z'],table['VI_CLASS'])
        title += '\n'
        title += r'RR:Z:{:1.3f},Class:{}'.format(table['RR_Z'],table['RR_CLASS'])
        title += '\n'
        title += r'QN:Z:{:1.3f},Class:{}'.format(table['QN_Z'],QN_CLASS)
        ax.set_title(title)
        m = table['spectra'].min()
        M = table['spectra'].max()
        ax.grid()
        ax.set_ylim(m-2,M+2)
        for il,l in enumerate(lines):
            lam_VI = absorber_IGM[l]*(1+table['VI_Z'])
            lam_QN = absorber_IGM[l]*(1+table['Z_LINES'][il])
            w = abs(wave-lam_QN)<100
            if w.sum()>0:
                m = table['spectra'].min()-1
                M = table['spectra'].max()+1
                ax.plot([lam_VI,lam_VI], [m,M],'r--',color='black')
                ax.plot([lam_QN,lam_QN], [m,M],'r--', alpha=0.1+0.9*table['C_LINES'][il])
                ax.text(lam_QN,M+0.5,'c$_{{{}}}={}$'.format(l,round(table['C_LINES'][il,],3)),
                         horizontalalignment='center',alpha=0.1+0.9*table['C_LINES'][il])
        patches = [mpatches.Patch(color='black', label='VI')]
        patches += [mpatches.Patch(color='red', label='QN')]
        ax.set_xlabel(r'$\lambda_\mathrm{obs}~[\AA]$')
        ax.set_ylabel(r'renormalised flux')
        ax.set_xlim(3600,10000)
        plt.legend(loc='lower left', handles=patches, fontsize=20)
        plt.show()
    return

In [ ]:
plot_spectrum(QN_false_z)

In [ ]:
#flux histogram 